In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
data_fake = pd.read_csv("Fake.csv")
data_true = pd.read_csv("True.csv")

In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
data_fake["class"] = 0
data_true["class"] = 1

In [6]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_fake_testing_manual = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i], axis = 0, inplace = True)
    
data_true_testing_manual = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i], axis = 0, inplace = True)

In [8]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [9]:
data_fake_testing_manual["class"] = 0
data_true_testing_manual["class"] = 1

C:\Users\dedak\AppData\Local\Temp\ipykernel_14024\2720973716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_testing_manual["class"] = 0
C:\Users\dedak\AppData\Local\Temp\ipykernel_14024\2720973716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_testing_manual["class"] = 1


In [10]:
data_fake_testing_manual.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [11]:
data_true_testing_manual.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [12]:
data_testing_manual = pd.concat([data_fake_testing_manual,data_true_testing_manual], axis = 0)
data_testing_manual.to_csv("testing.csv")

In [13]:
data = pd.concat([data_fake, data_true], axis =0 )
data.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [14]:
data.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [15]:
data = data.drop(["title", "subject","date"], axis = 1)

In [16]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [17]:
data = data.sample(frac = 1)

In [18]:
data.head()

,text,class
1789,If there s one thing Donald Trump has an extre...,0
2467,"SAVONLINNA, Finland (Reuters) - President Vlad...",1
2323,MOSCOW (Reuters) - Accusations made by U.S. Vi...,1
3252,Donald Trump is still using his Twitter accoun...,0
12040,NEW DELHI (Reuters) - A dozen Arab ambassadors...,1


In [19]:
data.reset_index(inplace = True)
data.drop(["index"], axis = 1, inplace = True)

In [20]:
data.columns

Index(['text', 'class'], dtype='object')

In [21]:
data.head()

,text,class
0,If there s one thing Donald Trump has an extre...,0
1,"SAVONLINNA, Finland (Reuters) - President Vlad...",1
2,MOSCOW (Reuters) - Accusations made by U.S. Vi...,1
3,Donald Trump is still using his Twitter accoun...,0
4,NEW DELHI (Reuters) - A dozen Arab ambassadors...,1


In [22]:
def Word_Processing(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [23]:
data["text"] = data["text"].apply(Word_Processing)

In [24]:
X = data["text"]
Y = data["class"]

In [25]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,Y, test_size=0.25)

In [26]:
vectorization = TfidfVectorizer()
X_Train_Vectorized = vectorization.fit_transform(X_Train)
X_Test_Vectorized = vectorization.transform(X_Test)

In [27]:

Logistic_Regression_Model = LogisticRegression()
Logistic_Regression_Model.fit(X_Train_Vectorized,Y_Train)

LogisticRegression()

In [28]:
Prediction_Logistic_Regreesion =Logistic_Regression_Model.predict(X_Test_Vectorized)

In [29]:
Logistic_Regression_Model.score(X_Test_Vectorized,Y_Test)

0.9872549019607844

In [30]:
print(classification_report(Y_Test,Prediction_Logistic_Regreesion))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5920
           1       0.98      0.99      0.99      5300

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [31]:
Decision_Tree_Classifier_Model = DecisionTreeClassifier()
Decision_Tree_Classifier_Model.fit(X_Train_Vectorized,Y_Train)

DecisionTreeClassifier()

In [32]:
Prediction_Decision_Tree_Classifier = Decision_Tree_Classifier_Model.predict(X_Test_Vectorized)

In [33]:
Decision_Tree_Classifier_Model.score(X_Test_Vectorized, Y_Test)

0.9953654188948307

In [34]:
print(classification_report(Y_Test, Prediction_Decision_Tree_Classifier))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5920
           1       1.00      0.99      1.00      5300

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [35]:
Gradient_Boosting_Classifier_Model = GradientBoostingClassifier(random_state=0)
Gradient_Boosting_Classifier_Model.fit(X_Train_Vectorized, Y_Train)

GradientBoostingClassifier(random_state=0)

In [36]:
Prediction_Gradient_Boosting_Classifier = Gradient_Boosting_Classifier_Model.predict(X_Test_Vectorized)

In [37]:
Gradient_Boosting_Classifier_Model.score(X_Test_Vectorized, Y_Test)

0.9948306595365419

In [38]:
print(classification_report(Y_Test, Prediction_Gradient_Boosting_Classifier))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5920
           1       0.99      1.00      0.99      5300

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [39]:
Random_Forest_Classifier_Model = RandomForestClassifier(random_state=0)
Random_Forest_Classifier_Model.fit(X_Train_Vectorized, Y_Train)

RandomForestClassifier(random_state=0)

In [40]:
Predict_Random_Forest_Classifier = Random_Forest_Classifier_Model.predict(X_Test_Vectorized)

In [41]:
Random_Forest_Classifier_Model.score(X_Test_Vectorized, Y_Test)

0.9903743315508021

In [42]:
print(classification_report(Y_Test, Predict_Random_Forest_Classifier))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5920
           1       0.99      0.99      0.99      5300

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [43]:
def Labelling(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def Testing_Manually(news):
    testing_news = {"text":[news]}
    new_data_test = pd.DataFrame(testing_news)
    new_data_test["text"] = new_data_test["text"].apply(Word_Processing) 
    X_Test_Data = new_data_test["text"]
    X_Test_Vectorized_Data = vectorization.transform(X_Test_Data)
    Prediction_Logistic_Regreesion = Logistic_Regression_Model.predict(X_Test_Vectorized_Data)
    Prediction_Decision_Tree_Classifier = Decision_Tree_Classifier_Model.predict(X_Test_Vectorized_Data)
    Prediction_Gradient_Boosting_Classifier = Gradient_Boosting_Classifier_Model.predict(X_Test_Vectorized_Data)
    Predict_Random_Forest_Classifier = Random_Forest_Classifier_Model.predict(X_Test_Vectorized_Data)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(Labelling(Prediction_Logistic_Regreesion[0]),                                                                                                    
                                                                                                              Labelling(Prediction_Decision_Tree_Classifier[0]), 
                                                                                                              Labelling(Prediction_Gradient_Boosting_Classifier[0]), 
                                                                                                              Labelling(Predict_Random_Forest_Classifier[0])))

In [44]:
News = str(input())
Testing_Manually(News)

SAO PAULO (Reuters) - Cesar Mata Pires, the owner and co-founder of Brazilian engineering conglomerate OAS SA, one of the largest companies involved in Brazil s corruption scandal, died on Tuesday. He was 68. Mata Pires died of a heart attack while taking a morning walk in an upscale district of S o Paulo, where OAS is based, a person with direct knowledge of the matter said. Efforts to contact his family were unsuccessful. OAS declined to comment. The son of a wealthy cattle rancher in the northeastern state of Bahia, Mata Pires  links to politicians were central to the expansion of OAS, which became Brazil s No. 4 builder earlier this decade, people familiar with his career told Reuters last year. His big break came when he befriended Antonio Carlos Magalh es, a popular politician who was Bahia governor several times, and eventually married his daughter Tereza. Brazilians joked that OAS stood for  Obras Arranjadas pelo Sogro  - or  Work Arranged by the Father-In-Law.   After years of

In [45]:
News = str(input())
Testing_Manually(News)

Vic Bishop Waking TimesOur reality is carefully constructed by powerful corporate, political and special interest sources in order to covertly sway public opinion. Blatant lies are often televised regarding terrorism, food, war, health, etc. They are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms.The practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this. The entire role of this brainwashing industry is to figure out how to spin information to journalists, similar to the lobbying of government. It is never really clear just how much truth the journalists receive because the news industry has become complacent. The messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own 90% of the media:General Electric (GE), News-Corp, Disney, Viacom, Time Warner, a